In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.integrate
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
import scipy.optimize
from matplotlib.font_manager import FontProperties
import matplotlib.colors as colors
import matplotlib.cbook as cbook
from matplotlib import cm
import pandas as pd
import copy
import matplotlib.ticker as mticker
from cycler import cycler
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
import func #this contains all functions
from matplotlib.ticker import FormatStrFormatter
import matplotlib as matplotlib
import itertools


### Parameters

In [ ]:
b = 1.5
t_w = 0.5 #waiting time for shuttles
nu = 1
ELL = [0.2,0.4,0.8] #ell values
E_list = [100,1000,10000] # nu E represents Lambda here nu = 1 so Lambda = E
vel_rat_list = [1.22,3.6,7.8] # for above Lambdas
ta_list = [0.058,0.055,0.04] #for above Lambdas
ts_list = ta_list #we set t_a = t_s
delta = 1.5 #delta = 1.5 as said in paper
ell_list = np.logspace(-1,2,100)
capacity = 100 #train capacity

### Supply Demand

In [ ]:
D_CUT = ell_list
index = 0
for E in E_list:
    ta = ta_list[index]
    ts = ts_list[index]
    vel_rat = vel_rat_list[index]
    for l in ELL:
        dldcFuni = func.get_dldcFuniVectorized(D_CUT)
        dgdcFbi = func.get_dgdcFbiVectorized(D_CUT)
        Funi = func.get_FuniVectorized(D_CUT)
        Fbi = 1 - Funi
        params = {"delta": delta,
                  "b": b,
                  "l": l,
                  "dc": D_CUT,
                  "alpha": 1,
                  "t_w": t_w,
                  "nu": nu,
                  "E": E,
                  "vel_rat": vel_rat,
                  "power_law": True,
                  "capacity": capacity,
                  "dldcFuni": dldcFuni,
                  "dgdcFbi": dgdcFbi,
                  "Funi": Funi,
                  "Fbi": Fbi,
                  "ta": ta,
                  "ts": ts,
        }
        eta_SD,emission_SD = func.getCarbonIndexVectorized(**params)
        convenience_SD = 1./func.getConvenienceVectorized(**params)

        convenience_SD = convenience_SD.ravel()
        emission_SD = emission_SD.ravel()
        eta_SD = eta_SD.ravel()

        #Compute Paretos
        pareto_SD = func.getParetoFront(convenience_SD,emission_SD)
        pareto_front_convenience_SD = convenience_SD[pareto_SD]
        pareto_front_emission_SD = emission_SD[pareto_SD]
        pareto_front_DCUT_SD = D_CUT[pareto_SD]
        pareto_front_eta_SD = eta_SD[pareto_SD]
        pareto_front_bimodal_SD = Fbi[pareto_SD]

        #Pareto data
        df_pareto_SD = pd.DataFrame()
        df_pareto_SD['quality'] = pareto_front_convenience_SD
        df_pareto_SD['emission'] = pareto_front_emission_SD
        df_pareto_SD['d_cut'] = pareto_front_DCUT_SD
        #df_pareto_SD['l'] = pareto_front_ELL_SD
        df_pareto_SD['eta'] = pareto_front_eta_SD
        df_pareto_SD['bimodal_frac'] = pareto_front_bimodal_SD
        #df_pareto_SD['capacity'] = pareto_front_capacity_SD
        df_pareto_SD.sort_values('quality', inplace=True)
        df_pareto_SD.to_csv('data/Pareto_SupplyDemand%d_l_%0.2f.csv'%(E,l))

        #Scatter data
        df_scatter_SD = pd.DataFrame()
        df_scatter_SD['quality'] = convenience_SD
        df_scatter_SD['emission'] = emission_SD
        df_scatter_SD['d_cut'] = D_CUT
        #df_scatter_SD['l'] = flat_ELL
        df_scatter_SD.sort_values('quality', inplace=True)
        df_scatter_SD.to_csv('data/Scatter_SupplyDemand%d_l_%0.2f.csv'%(E,l))
        
    index+=1

    

### No Supply Demand

In [ ]:
alpha_list = np.logspace(-2,0,20)
d_cut_list = ell_list
d_cut_mesh,alpha_mesh = np.meshgrid(d_cut_list,alpha_list)
D_CUT = d_cut_mesh.ravel()
alpha = alpha_mesh.ravel()
index = 0
for E in E_list:
    ta = ta_list[index]
    ts = ts_list[index]
    vel_rat = vel_rat_list[index]
    for l in ELL:
        dldcFuni = func.get_dldcFuniVectorized(D_CUT)
        dgdcFbi = func.get_dgdcFbiVectorized(D_CUT)
        Funi = func.get_FuniVectorized(D_CUT)
        Fbi = 1 - Funi
        params = {"delta": delta,
                  "b": b,
                  "l": l,
                  "dc": D_CUT,
                  "alpha": alpha,
                  "t_w": t_w,
                  "nu": nu,
                  "E": E,
                  "vel_rat": vel_rat,
                  "power_law": True,
                  "capacity": capacity,
                  "dldcFuni": dldcFuni,
                  "dgdcFbi": dgdcFbi,
                  "Funi": Funi,
                  "Fbi": Fbi,
                  "ta": ta,
                  "ts": ts,
        }
        eta_noSD,emission_noSD = func.getCarbonIndexVectorized(**params)
        convenience_noSD = 1./func.getConvenienceVectorized(**params)

        convenience_noSD = convenience_noSD.ravel()
        emission_noSD = emission_noSD.ravel()
        eta_noSD = eta_noSD.ravel()

        #Compute Paretos
        pareto_noSD = func.getParetoFront(convenience_noSD,emission_noSD)
        pareto_front_convenience_noSD = convenience_noSD[pareto_noSD]
        pareto_front_emission_noSD = emission_noSD[pareto_noSD]
        pareto_front_DCUT_noSD = D_CUT[pareto_noSD]
        pareto_front_eta_noSD = eta_noSD[pareto_noSD]
        pareto_front_bimodal_noSD = Fbi[pareto_noSD]
        pareto_front_alpha_noSD = alpha[pareto_noSD]

        #Pareto data
        df_pareto_noSD = pd.DataFrame()
        df_pareto_noSD['quality'] = pareto_front_convenience_noSD
        df_pareto_noSD['emission'] = pareto_front_emission_noSD
        df_pareto_noSD['d_cut'] = pareto_front_DCUT_noSD
        df_pareto_noSD['eta'] = pareto_front_eta_noSD
        df_pareto_noSD['bimodal_frac'] = pareto_front_bimodal_noSD
        df_pareto_noSD['alpha'] = pareto_front_alpha_noSD
        df_pareto_noSD.sort_values('quality', inplace=True)
        df_pareto_noSD.to_csv('data/Pareto_noSupplyDemand_%d_l_%0.2f.csv'%(E,l))
        
        #Scatter datacd ps
        df_scatter_noSD = pd.DataFrame()
        df_scatter_noSD['quality'] = convenience_noSD
        df_scatter_noSD['emission'] = emission_noSD
        df_scatter_noSD['d_cut'] = D_CUT
        df_scatter_noSD['alpha'] = alpha
        df_scatter_noSD.sort_values('quality', inplace=True)
        df_scatter_noSD.to_csv('data/Scatter_noSupplyDemand%d_l_%0.2f.csv'%(E,l))
        
    index+=1